In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%env OPENCV_VIDEOIO_PRIORITY_MSMF 0

env: OPENCV_VIDEOIO_PRIORITY_MSMF=0


In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
face_classifier = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

In [4]:
mask_detect_model = keras.models.load_model('best_model.h5')

In [5]:
result = {0:"Mask",1:"No Mask"}
bgr_color = {0:(0,255,0),1:(0,0,255)}

In [16]:
def detect_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.medianBlur(gray_frame,13)
    face_rects = face_classifier.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)
    
    for x,y,w,h in face_rects:
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.resize(face_frame,(128,128))
        
        face_frame = face_frame/255.0
        face_frame = np.expand_dims(face_frame,axis=0)
        
        detect = mask_detect_model.predict_classes(face_frame)
        label = detect[0,0]
        
        cv2.putText(frame,result[label],(x-10,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,bgr_color[label],3)
        cv2.rectangle(frame,(x,y),(x+w,y+h),bgr_color[label],2)
    
    return frame

In [17]:
capture = cv2.VideoCapture(0)

while True:
    ret,frame = capture.read()
    
    if not ret:
        break
    
    frame = cv2.flip(frame,1)
    
    final_frame = detect_faces(frame)
    cv2.imshow('Faces', final_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()